# Hadoop HDFS upload data

### Update sys requirements

In [3]:
!sudo apt update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
1 package can be upgraded. Run 'apt list --upgradable' to see it.


In [24]:
!sudo apt install libsasl2-dev libsasl2-2 libsasl2-modules-gssapi-mit sasl2-bin libsasl2-2 libsasl2-dev libsasl2-modules -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsasl2-2 is already the newest version (2.1.27+dfsg2-3ubuntu1.2).
libsasl2-dev is already the newest version (2.1.27+dfsg2-3ubuntu1.2).
libsasl2-modules is already the newest version (2.1.27+dfsg2-3ubuntu1.2).
libsasl2-modules-gssapi-mit is already the newest version (2.1.27+dfsg2-3ubuntu1.2).
sasl2-bin is already the newest version (2.1.27+dfsg2-3ubuntu1.2).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [17]:
!pip install sasl thrift hdfs PyHive thrift_sasl

### Upload dataset to HDFS

In [45]:
import os
from pyhive import hive
import pandas as pd
from io import StringIO
from hdfs import InsecureClient

In [58]:
client = InsecureClient('http://namenode:9870', user='root')

# Define the paths for the tables and views
stocks_hdfs_path = '/data/opendan/files/Stocks'

In [59]:
for filename in os.listdir('Stocks'):
    try:
        df = pd.read_csv(f'Stocks/{filename}')
        name = filename.split('.')[0]
        df['Name'] = name
        hdfs_dest = os.path.join(stocks_hdfs_path, name + '.csv')
        # Save the tables as CSV files to HDFS
        with client.write(hdfs_dest, encoding='utf-8', overwrite=True) as writer:
            df.to_csv(writer, index=False)
    except:
        # its ok, some stock data is empty or broken (old dataset)
        print(f'Error with {filename}')